In [75]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [76]:
import pandas as pd
df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/chatbot deva3/nexyug_chatbot_dataset_3000.xlsx')

In [77]:
df.isnull().sum()

,0
User Input,0
Bot Response,0
Intent,0


In [78]:
df["User Input"]=df["User Input"].str.lower()
df["Bot Response"]=df["Bot Response"].str.lower()
df["Intent"]=df["Intent"].str.lower()
df.head(2)

,User Input,Bot Response,Intent
0,hi,hello! how can i assist you today?,greeting
1,hello,hi there! how can i help you?,greeting


In [79]:
pattern_to_remove = r'#\d+'
df["User Input"]=df["User Input"].str.replace(pattern_to_remove, '', regex=True)
df["User Input"]=df["User Input"].str.replace(r'[^\w\s]', '', regex=True)
df.head(2)

,User Input,Bot Response,Intent
0,hi,hello! how can i assist you today?,greeting
1,hello,hi there! how can i help you?,greeting


In [80]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [81]:
df['User Input'] = df['User Input'].fillna('').astype(str)
df['tokenized_user_input'] = df['User Input'].apply(word_tokenize)
df.head()

,User Input,Bot Response,Intent,tokenized_user_input
0,hi,hello! how can i assist you today?,greeting,[hi]
1,hello,hi there! how can i help you?,greeting,[hello]
2,hey,hey! what can i do for you?,greeting,[hey]
3,good morning,good morning! how may i help you today?,greeting,"[good, morning]"
4,good evening,good evening! what can i assist you with?,greeting,"[good, evening]"


In [82]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [83]:
stop_words

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 "he'd",
 "he'll",
 "he's",
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 "i'd",
 "i'll",
 "i'm",
 "i've",
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it'd",
 "it'll",
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'on

In [84]:
def remove_stopwords_from_list(tokens):
    return [word for word in tokens if word not in stop_words]

df['filtered_user_input'] = df['tokenized_user_input'].apply(remove_stopwords_from_list)
df.head()

,User Input,Bot Response,Intent,tokenized_user_input,filtered_user_input
0,hi,hello! how can i assist you today?,greeting,[hi],[hi]
1,hello,hi there! how can i help you?,greeting,[hello],[hello]
2,hey,hey! what can i do for you?,greeting,[hey],[hey]
3,good morning,good morning! how may i help you today?,greeting,"[good, morning]","[good, morning]"
4,good evening,good evening! what can i assist you with?,greeting,"[good, evening]","[good, evening]"


In [85]:
dataset = []

for _, row in df.iterrows():
    user_input = " ".join(row['filtered_user_input'])
    response = row['Bot Response']
    dataset.append({
        "input": user_input,
        "output": response
    })

In [86]:
dataset[0:10]

[{'input': 'hi', 'output': 'hello! how can i assist you today?'},
 {'input': 'hello', 'output': 'hi there! how can i help you?'},
 {'input': 'hey', 'output': 'hey! what can i do for you?'},
 {'input': 'good morning',
  'output': 'good morning! how may i help you today?'},
 {'input': 'good evening',
  'output': 'good evening! what can i assist you with?'},
 {'input': 'whats', 'output': 'hey! how can i support you today?'},
 {'input': 'anyone', 'output': "yes, i'm here to help you."},
 {'input': 'hi team', 'output': 'hi! how can our team assist you?'},
 {'input': 'greetings',
  'output': 'greetings! what service are you looking for?'},
 {'input': 'yo', 'output': 'yo! what can i do for you today?'}]

In [87]:
import json
with open("chatbot_data.json", "w") as f:
    json.dump(dataset, f, indent=2)

In [88]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments

tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained("gpt2")

In [89]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [90]:
input_text ="be professional dude"
input_ids = tokenizer.encode(input_text, return_tensors="pt")

output = model.generate(input_ids, max_length=38)
tokenizer.decode(output[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


"be professional dude.\n\nI'm not sure if I'm going to be able to do this, but I'm going to be able to do it. I'm going to be able"

In [91]:
from datasets import Dataset
df = pd.read_json("/content/chatbot_data.json")
dataset = Dataset.from_pandas(df)

In [92]:
dataset, df

(Dataset({
     features: ['input', 'output'],
     num_rows: 450
 }),
                                    input  \
 0                                     hi   
 1                                  hello   
 2                                    hey   
 3                           good morning   
 4                           good evening   
 ..                                   ...   
 445  whatsapp par message bhej sakte hai   
 446              whatsapp handle kya hai   
 447           whatsapp support available   
 448     whatsapp se contact kaise karein   
 449            message whatsapp possible   
 
                                                 output  
 0                   hello! how can i assist you today?  
 1                        hi there! how can i help you?  
 2                          hey! what can i do for you?  
 3              good morning! how may i help you today?  
 4            good evening! what can i assist you with?  
 ..                                    

In [93]:
def tokenize_function(example):
    prompt = f"User: {example['input']}\nBot:"
    full_text = prompt + " " + example["output"]

    tokens = tokenizer(
        full_text,
        padding="max_length",
        truncation=True,
        max_length=128
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

In [94]:
tokenized = dataset.map(tokenize_function, remove_columns=dataset.column_names)

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

In [95]:
tokenized['labels']

Column([[12982, 25, 23105, 198, 20630, 25, 23748, 0, 703, 460, 1312, 3342, 345, 1909, 30, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256], [12982, 25, 23748, 198, 20630, 25, 23105, 612, 0, 703, 460, 1312, 1037, 345, 30, 50256, 50256, 50256, 50256, 50256, 50

In [96]:
import torch
from transformers import (
    GPT2TokenizerFast,
    GPT2LMHeadModel,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
)

In [97]:
use_cuda = torch.cuda.is_available()
use_cuda

True

In [98]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [99]:
training_args = TrainingArguments(
    output_dir="./gpt2-chatbot",
    per_device_train_batch_size=4,
    num_train_epochs=4,
    save_strategy="epoch",
    logging_dir="./logs",
    fp16=use_cuda,              # ← enable half-precision only on CUDA GPUs
    push_to_hub=False,          # optional: disable if you aren’t pushing to HF Hub
)

In [100]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized,
    eval_dataset=tokenized,     # replace with your eval split
    tokenizer=tokenizer,        # still needed for generation, etc.
    data_collator=data_collator # fixes the FutureWarning
)

/tmp/ipython-input-480345068.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [101]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=452, training_loss=0.5987796614655351, metrics={'train_runtime': 101.6998, 'train_samples_per_second': 17.699, 'train_steps_per_second': 4.444, 'total_flos': 117581414400000.0, 'train_loss': 0.5987796614655351, 'epoch': 4.0})

In [102]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [103]:
def generate_response(user_input):
    prompt = f"User: {user_input}\nBot:"
    inputs = tokenizer(prompt, return_tensors="pt", padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    outputs = model.generate(
        **inputs,
        max_length=128,
        num_return_sequences=1,
        pad_token_id=tokenizer.pad_token_id
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.split("Bot:")[-1].strip()

In [109]:
user_input= input("Query: ")
generate_response(user_input)

Query: software 


'yes, we build and optimize software for businesses.'

In [111]:
trainer.save_model()